# Split the codebook on training and validation parts

In [1]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers.core import Dense, Lambda
from keras.layers import LSTM, Embedding, Bidirectional, BatchNormalization, SimpleRNN, Input, TimeDistributed
from keras import backend as K
import matplotlib.pyplot as plt
from tqdm.keras import TqdmCallback
from scipy.spatial import distance
%matplotlib inline
%config InlineBackend.figure_format='retina'
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]

In [2]:
k = 8                       # number of information bits
N = 16                      # code length

nb_epoch = 2**18 
batch_size = 256            # size of batches for calculation the gradient
LLR = False                 # 'True' enables the log-likelihood-ratio layer
noise = True
optimizer = 'adam'           
loss = 'mse'                # or 'binary_crossentropy'

train_SNR_Eb = 1            # training-Eb/No
train_SNR_Es = train_SNR_Eb + 10*np.log10(k/N)
train_sigma = np.sqrt(1/(2*10**(train_SNR_Es/10))).tolist()

In [3]:
def modulateBPSK(x):
    return -2*x +1;

def addNoise(x, sigma):
    w = K.random_normal(K.shape(x), mean=0.0, stddev=sigma)        # std was changed to stddev 
    return x + w

def ber(y_true, y_pred):
    return K.mean(K.not_equal(y_true, K.round(y_pred)))    

def return_output_shape(input_shape):  
    return input_shape

def compose_model(layers):
    model = Sequential()
    for layer in layers:
        model.add(layer)
    return model

def log_likelihood_ratio(x, sigma):   
    return 2*x/np.float32(sigma**2)

def errors(y_true, y_pred):
    return K.sum(int(K.not_equal(y_true, K.round(y_pred))))

def reshape_codebook(cb):
    global N
    global output_shape_test
    output_shape_test = tf.reshape(cb, (-1, N, 1))
    return output_shape_test

output_shape_test = (0, 0, 0)

In [4]:
def update_model(current_arch):
    # Define modulator
    modulator_layers = [Lambda(modulateBPSK, 
                              input_shape=(N,), output_shape=return_output_shape, name="modulator")]
    modulator = compose_model(modulator_layers)
    modulator.compile(optimizer=optimizer, loss=loss)

    # Define noise
    noise_layers = [Lambda(addNoise, arguments={'sigma':train_sigma}, 
                           input_shape=(N,), output_shape=return_output_shape, name="noise")]
    noise = compose_model(noise_layers)
    noise.compile(optimizer=optimizer, loss=loss)
    
    # Reshape layer
    reshape_layer = [Lambda(reshape_codebook,
                            input_shape = (N,), output_shape = output_shape_test, name = 'reshape')]
    reshape = compose_model(reshape_layer)
    reshape.compile(optimizer=optimizer, loss=loss)

    # Define LLR
    llr_layers = [Lambda(log_likelihood_ratio, arguments={'sigma':train_sigma}, 
                         input_shape=(N,), output_shape=return_output_shape, name="LLR")]
    llr = compose_model(llr_layers)
    llr.compile(optimizer=optimizer, loss=loss)
    
    
    # Define decoder
    
    if current_arch == 'dense-128-64-32':
        decoder_layers = [Dense(128, activation='relu'),
                          Dense(64, activation='relu'),
                          Dense(32, activation='relu'),
                          Dense(k, activation='sigmoid')]
        decoder = compose_model(decoder_layers)
        decoder.compile(optimizer=optimizer, loss=loss, metrics=[errors])
        
    elif current_arch == 'dense-256-128-64-32':
        decoder_layers = [Dense(256, activation='relu', input_shape=(N,)),
                          Dense(128, activation='relu'),
                          Dense(64, activation='relu'),
                          Dense(32, activation='relu'),
                          Dense(k, activation='sigmoid')]
        decoder = compose_model(decoder_layers)
        decoder.compile(optimizer=optimizer, loss=loss, metrics=[errors])
        
    elif current_arch == 'lstm-32':
        decoder_layers = [Lambda(reshape_codebook,
                            input_shape = (N,), output_shape = output_shape_test, name = 'reshape'),
                          LSTM(32, activation='relu'),
                          Dense(k, activation='sigmoid')]
        decoder = compose_model(decoder_layers)
        decoder.compile(optimizer=optimizer, loss=loss, metrics=[errors])
        
    elif current_arch == 'lstm-64':
        decoder_layers = [Lambda(reshape_codebook,
                            input_shape = (N,), output_shape = output_shape_test, name = 'reshape'),
                          LSTM(64, activation='relu'),
                          Dense(k, activation='sigmoid')]
        decoder = compose_model(decoder_layers)
        decoder.compile(optimizer=optimizer, loss=loss, metrics=[errors])
        
    elif current_arch == 'lstm-128':
        decoder_layers = [Lambda(reshape_codebook,
                            input_shape = (N,), output_shape = output_shape_test, name = 'reshape'),
                          LSTM(128, activation='relu'),
                          Dense(k, activation='sigmoid')]
        decoder = compose_model(decoder_layers)
        decoder.compile(optimizer=optimizer, loss=loss, metrics=[errors])
        
    elif current_arch == 'lstm-256':
        decoder_layers = [Lambda(reshape_codebook,
                            input_shape = (N,), output_shape = output_shape_test, name = 'reshape'),
                          LSTM(256, activation='relu'),
                          Dense(k, activation='sigmoid')]
        decoder = compose_model(decoder_layers)
        decoder.compile(optimizer=optimizer, loss=loss, metrics=[errors])
        
        
    elif current_arch == 'lstm-64-32':
        decoder_layers = [Lambda(reshape_codebook,
                            input_shape = (N,), output_shape = output_shape_test, name = 'reshape'),
                          LSTM(64, activation='relu', return_sequences=True), 
                          LSTM(32, activation='relu'),
                          Dense(k, activation='sigmoid')]
        decoder = compose_model(decoder_layers)
        decoder.compile(optimizer=optimizer, loss=loss, metrics=[errors])
        
        
    elif current_arch == 'lstm-128-64-32':
        decoder_layers = [Lambda(reshape_codebook,
                            input_shape = (N,), output_shape = output_shape_test, name = 'reshape'),
                          LSTM(128, activation='relu', return_sequences=True), 
                          LSTM(64, activation='relu', return_sequences=True), 
                          LSTM(32, activation='relu'),
                          Dense(k, activation='sigmoid')]
        decoder = compose_model(decoder_layers)
        decoder.compile(optimizer=optimizer, loss=loss, metrics=[errors])
        
    elif current_arch == 'lstm-256-128-64-32':
        decoder_layers = [Lambda(reshape_codebook,
                            input_shape = (N,), output_shape = output_shape_test, name = 'reshape'),
                          LSTM(256, activation='relu', return_sequences=True),
                          LSTM(128, activation='relu', return_sequences=True),
                          LSTM(64, activation='relu', return_sequences=True), 
                          LSTM(32, activation='relu'),
                          Dense(k, activation='sigmoid')]
        decoder = compose_model(decoder_layers)
        decoder.compile(optimizer=optimizer, loss=loss, metrics=[errors])


    # Define model
#     if (LLR and noise):
#         model_layers = modulator_layers + noise_layers  + llr_layers + decoder_layers
#     elif (not LLR and noise):
#         model_layers = modulator_layers + noise_layers + decoder_layers
#     elif (not LLR and not noise):
#         model_layers = modulator_layers + decoder_layers

    if noise:
        model_layers = modulator_layers + noise_layers + decoder_layers
    else:
        model_layers = modulator_layers + decoder_layers
    model = compose_model(model_layers)
    model.compile(optimizer=optimizer, loss=loss, metrics=[ber])
    return model, decoder

In [5]:
def half_adder(a,b):
    s = a ^ b
    c = a & b
    return s,c

def full_adder(a,b,c):
    s = (a ^ b) ^ c
    c = (a & b) | (c & (a ^ b))
    return s,c

def add_bool(a,b):
    if len(a) != len(b):
        raise ValueError('arrays with different length')
    k = len(a)
    s = np.zeros(k,dtype=bool)
    c = False
    for i in reversed(range(0,k)):
        s[i], c = full_adder(a[i],b[i],c)    
    if c:
        warnings.warn("Addition overflow!")
    return s

def inc_bool(a):
    k = len(a)
    increment = np.hstack((np.zeros(k-1,dtype=bool), np.ones(1,dtype=bool)))
    a = add_bool(a,increment)
    return a

def bitrevorder(x):
    m = np.amax(x)
    n = np.ceil(np.log2(m)).astype(int)
    for i in range(0,len(x)):
        x[i] = int('{:0{n}b}'.format(x[i],n=n)[::-1],2)  
    return x

def int2bin(x,N):
    if isinstance(x, list) or isinstance(x, np.ndarray):
        binary = np.zeros((len(x),N),dtype='bool')
        for i in range(0,len(x)):
            binary[i] = np.array([int(j) for j in bin(x[i])[2:].zfill(N)])
    else:
        binary = np.array([int(j) for j in bin(x)[2:].zfill(N)],dtype=bool)
    
    return binary

def bin2int(b):
    if isinstance(b[0], list):
        integer = np.zeros((len(b),),dtype=int)
        for i in range(0,len(b)):
            out = 0
            for bit in b[i]:
                out = (out << 1) | bit
            integer[i] = out
    elif isinstance(b, np.ndarray):
        if len(b.shape) == 1:
            out = 0
            for bit in b:
                out = (out << 1) | bit
            integer = out     
        else:
            integer = np.zeros((b.shape[0],),dtype=int)
            for i in range(0,b.shape[0]):
                out = 0
                for bit in b[i]:
                    out = (out << 1) | bit
                integer[i] = out
        
    return integer

def polar_design_awgn(N, k, design_snr_dB):  
        
    S = 10**(design_snr_dB/10)
    z0 = np.zeros(N)

    z0[0] = np.exp(-S)
    for j in range(1,int(np.log2(N))+1):
        u = 2**j
        for t in range(0,int(u/2)):
            T = z0[t]
            z0[t] = 2*T - T**2     # upper channel
            z0[int(u/2)+t] = T**2  # lower channel
        
    # sort into increasing order
    idx = np.argsort(z0)
        
    # select k best channels
    idx = np.sort(bitrevorder(idx[0:k]))
    
    A = np.zeros(N, dtype=bool)
    A[idx] = True
        
    return A

def polar_transform_iter(u):

    N = len(u)
    n = 1
    x = np.copy(u)
    stages = np.log2(N).astype(int)
    for s in range(0,stages):
        i = 0
        while i < N:
            for j in range(0,n):
                idx = i+j
                x[idx] = x[idx] ^ x[idx+n]
            i=i+2*n
        n=2*n
    return x

In [6]:
def create_words(code):
    # Create all possible information words
    d = np.zeros((2**k,k),dtype=bool)
    for i in range(1,2**k):
        d[i]= inc_bool(d[i-1])

    # Create sets of all possible codewords (codebook)
    if code == 'polar':   

        A = polar_design_awgn(N, k, design_snr_dB=0)  # logical vector indicating the nonfrozen bit locations 
        x = np.zeros((2**k, N),dtype=bool)
        u = np.zeros((2**k, N),dtype=bool)
        u[:,A] = d

        for i in range(0,2**k):
            x[i] = polar_transform_iter(u[i])
        return x, d, A

    elif code == 'random':

        np.random.seed(4267)   # for a 16bit Random Code (r=0.5) with Hamming distance >= 2
        x = np.random.randint(0,2,size=(2**k,N), dtype=bool)
        return x, d

### Create codebook

In [7]:
code = 'polar'              # type of code ('random' or 'polar')
codewords, inputs, log_vector = create_words(code)

# With noise

In [8]:
noise = True

## i*train_size+i, train_size = 16

### Sptit codebook on train / validation parts

In [9]:
train_size = 16
idx = []
for i in range(train_size):
    idx.append(i*train_size+i)
print(f'training indices: {idx}')
inputs_train = inputs[idx]
codewords_train = codewords[idx]
d_test = np.delete(inputs, idx, axis=0)

training indices: [0, 17, 34, 51, 68, 85, 102, 119, 136, 153, 170, 187, 204, 221, 238, 255]


In [10]:
inputs_train.astype(int)[:16]

array([[0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 1, 0],
       [0, 0, 1, 1, 0, 0, 1, 1],
       [0, 1, 0, 0, 0, 1, 0, 0],
       [0, 1, 0, 1, 0, 1, 0, 1],
       [0, 1, 1, 0, 0, 1, 1, 0],
       [0, 1, 1, 1, 0, 1, 1, 1],
       [1, 0, 0, 0, 1, 0, 0, 0],
       [1, 0, 0, 1, 1, 0, 0, 1],
       [1, 0, 1, 0, 1, 0, 1, 0],
       [1, 0, 1, 1, 1, 0, 1, 1],
       [1, 1, 0, 0, 1, 1, 0, 0],
       [1, 1, 0, 1, 1, 1, 0, 1],
       [1, 1, 1, 0, 1, 1, 1, 0],
       [1, 1, 1, 1, 1, 1, 1, 1]])

### Fit model

In [ ]:
%%time
model, decoder = update_model('dense-128-64-32')
history = model.fit(codewords_train, inputs_train, batch_size=batch_size, epochs=nb_epoch,
                    shuffle=True, verbose=0, callbacks=[TqdmCallback(verbose=0)])

2023-02-27 18:44:50.134089: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-27 18:44:51.129711: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10349 MB memory:  -> device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:65:00.0, compute capability: 6.1
2023-02-27 18:44:51.130506: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 10407 MB memory:  -> device: 1, name: GeForce GTX 1080 Ti, pci bus id: 0000:b3:00.0, compute capability: 6.1


0epoch [00:00, ?epoch/s]

2023-02-27 18:44:51.703276: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


### Evaluate model

In [ ]:
test_batch = 1000000

SNR_dB_start_Eb = 0
SNR_dB_stop_Eb = 10
SNR_points = 20

SNR_dB_start_Es = SNR_dB_start_Eb + 10*np.log10(k/N)
SNR_dB_stop_Es = SNR_dB_stop_Eb + 10*np.log10(k/N)

sigma_start = np.sqrt(1/(2*10**(SNR_dB_start_Es/10)))
sigma_stop = np.sqrt(1/(2*10**(SNR_dB_stop_Es/10)))
sigmas = np.linspace(sigma_start, sigma_stop, SNR_points)

nb_errors = np.zeros((1,2**k,len(sigmas)),dtype=int)
nb_bits = np.zeros((1,2**k,len(sigmas)),dtype=int)
exp_descr = []

In [ ]:
exp_descr.append('i*train_size+i')           # Add legend

for i in range(0,len(sigmas)):

    x_test = np.zeros((d_test.shape[0], N), dtype=bool)
    u_test = np.zeros((d_test.shape[0], N), dtype=bool)

    u_test[:, log_vector] = d_test
    for ii in range(0,d_test.shape[0]):

        print(f'Decoding SNR: {i+1}/{len(sigmas)}, CW: {ii+1}/{d_test.shape[0]}   ', end="\r")

        x_test[ii] = polar_transform_iter(u_test[ii])
        d_test_batch = np.tile(d_test[ii], (test_batch, 1))
        x_test_batch = np.tile(x_test[ii], (test_batch, 1))


        # Modulator (BPSK)
        s_test = -2*x_test_batch + 1

        # Channel (AWGN)
        y_test = s_test + sigmas[i]*np.random.standard_normal(s_test.shape)


        nb_bits[nb_bits.shape[0]-1][ii][i] += d_test_batch.size
        nb_errors[nb_errors.shape[0]-1][ii][i] += decoder.evaluate(y_test, d_test_batch, batch_size=test_batch, verbose=0)[1]

## Take first 16 elements

### Sptit codebook on train / validation parts

In [ ]:
train_size = 16
idx = list(np.arange(train_size, dtype=int))
print(f'training indices: {idx}')
inputs_train = inputs[idx]
codewords_train = codewords[idx]
d_test = np.delete(inputs, idx, axis=0)

### Fit model

In [ ]:
%%time
model, decoder = update_model('dense-128-64-32')
history = model.fit(codewords_train, inputs_train, batch_size=batch_size, epochs=nb_epoch,
                    shuffle=True, verbose=0, callbacks=[TqdmCallback(verbose=0)])

### Evaluate model

In [ ]:
exp_descr.append('first 16 elements')                                          # Add legend
nb_errors = np.append(nb_errors, np.zeros((1, 2**k, len(sigmas))), axis=0)     # Add row for new data
nb_bits = np.append(nb_bits, np.zeros((1, 2**k, len(sigmas))), axis=0)     # Add row for new data

for i in range(0,len(sigmas)):

    x_test = np.zeros((d_test.shape[0], N), dtype=bool)
    u_test = np.zeros((d_test.shape[0], N), dtype=bool)

    u_test[:, log_vector] = d_test
    for ii in range(0,d_test.shape[0]):

        print(f'Decoding SNR: {i+1}/{len(sigmas)}, CW: {ii+1}/{d_test.shape[0]}   ', end="\r")

        x_test[ii] = polar_transform_iter(u_test[ii])
        d_test_batch = np.tile(d_test[ii], (test_batch, 1))
        x_test_batch = np.tile(x_test[ii], (test_batch, 1))


        # Modulator (BPSK)
        s_test = -2*x_test_batch + 1

        # Channel (AWGN)
        y_test = s_test + sigmas[i]*np.random.standard_normal(s_test.shape)


        nb_bits[nb_bits.shape[0]-1][ii][i] += d_test_batch.size
        nb_errors[nb_errors.shape[0]-1][ii][i] += decoder.evaluate(y_test, d_test_batch, batch_size=test_batch, verbose=0)[1]

## Take first 32 elements

### Sptit codebook on train / validation parts

In [ ]:
train_size = 32
idx = list(np.arange(train_size, dtype=int))
print(f'training indices: {idx}')
inputs_train = inputs[idx]
codewords_train = codewords[idx]
d_test = np.delete(inputs, idx, axis=0)

### Fit model

In [ ]:
%%time
model, decoder = update_model('dense-128-64-32')
history = model.fit(codewords_train, inputs_train, batch_size=batch_size, epochs=nb_epoch,
                    shuffle=True, verbose=0, callbacks=[TqdmCallback(verbose=0)])

### Evaluate model

In [ ]:
exp_descr.append('first 32 elements')                                          # Add legend
nb_errors = np.append(nb_errors, np.zeros((1, 2**k, len(sigmas))), axis=0)     # Add row for new data
nb_bits = np.append(nb_bits, np.zeros((1, 2**k, len(sigmas))), axis=0)     # Add row for new data

for i in range(0,len(sigmas)):

    x_test = np.zeros((d_test.shape[0], N), dtype=bool)
    u_test = np.zeros((d_test.shape[0], N), dtype=bool)

    u_test[:, log_vector] = d_test
    for ii in range(0,d_test.shape[0]):

        print(f'Decoding SNR: {i+1}/{len(sigmas)}, CW: {ii+1}/{d_test.shape[0]}   ', end="\r")

        x_test[ii] = polar_transform_iter(u_test[ii])
        d_test_batch = np.tile(d_test[ii], (test_batch, 1))
        x_test_batch = np.tile(x_test[ii], (test_batch, 1))


        # Modulator (BPSK)
        s_test = -2*x_test_batch + 1

        # Channel (AWGN)
        y_test = s_test + sigmas[i]*np.random.standard_normal(s_test.shape)


        nb_bits[nb_bits.shape[0]-1][ii][i] += d_test_batch.size
        nb_errors[nb_errors.shape[0]-1][ii][i] += decoder.evaluate(y_test, d_test_batch, batch_size=test_batch, verbose=0)[1]

## Take first 64 elements

### Sptit codebook on train / validation parts

In [ ]:
train_size = 64
idx = list(np.arange(train_size, dtype=int))
print(f'training indices: {idx}')
inputs_train = inputs[idx]
codewords_train = codewords[idx]
d_test = np.delete(inputs, idx, axis=0)

### Fit model

In [ ]:
%%time
model, decoder = update_model('dense-128-64-32')
history = model.fit(codewords_train, inputs_train, batch_size=batch_size, epochs=nb_epoch,
                    shuffle=True, verbose=0, callbacks=[TqdmCallback(verbose=0)])

### Evaluate model

In [ ]:
exp_descr.append('first 64 elements')                                          # Add legend
nb_errors = np.append(nb_errors, np.zeros((1, 2**k, len(sigmas))), axis=0)     # Add row for new data
nb_bits = np.append(nb_bits, np.zeros((1, 2**k, len(sigmas))), axis=0)     # Add row for new data

for i in range(0,len(sigmas)):

    x_test = np.zeros((d_test.shape[0], N), dtype=bool)
    u_test = np.zeros((d_test.shape[0], N), dtype=bool)

    u_test[:, log_vector] = d_test
    for ii in range(0,d_test.shape[0]):

        print(f'Decoding SNR: {i+1}/{len(sigmas)}, CW: {ii+1}/{d_test.shape[0]}   ', end="\r")

        x_test[ii] = polar_transform_iter(u_test[ii])
        d_test_batch = np.tile(d_test[ii], (test_batch, 1))
        x_test_batch = np.tile(x_test[ii], (test_batch, 1))


        # Modulator (BPSK)
        s_test = -2*x_test_batch + 1

        # Channel (AWGN)
        y_test = s_test + sigmas[i]*np.random.standard_normal(s_test.shape)


        nb_bits[nb_bits.shape[0]-1][ii][i] += d_test_batch.size
        nb_errors[nb_errors.shape[0]-1][ii][i] += decoder.evaluate(y_test, d_test_batch, batch_size=test_batch, verbose=0)[1]

## Take first 128 elements

### Sptit codebook on train / validation parts

In [ ]:
train_size = 128
idx = list(np.arange(train_size, dtype=int))
print(f'training indices: {idx}')
inputs_train = inputs[idx]
codewords_train = codewords[idx]
d_test = np.delete(inputs, idx, axis=0)

### Fit model

In [ ]:
%%time
model, decoder = update_model('dense-128-64-32')
history = model.fit(codewords_train, inputs_train, batch_size=batch_size, epochs=nb_epoch,
                    shuffle=True, verbose=0, callbacks=[TqdmCallback(verbose=0)])

### Evaluate model

In [ ]:
exp_descr.append('first 128 elements')                                          # Add legend
nb_errors = np.append(nb_errors, np.zeros((1, 2**k, len(sigmas))), axis=0)     # Add row for new data
nb_bits = np.append(nb_bits, np.zeros((1, 2**k, len(sigmas))), axis=0)     # Add row for new data

for i in range(0,len(sigmas)):

    x_test = np.zeros((d_test.shape[0], N), dtype=bool)
    u_test = np.zeros((d_test.shape[0], N), dtype=bool)

    u_test[:, log_vector] = d_test
    for ii in range(0,d_test.shape[0]):

        print(f'Decoding SNR: {i+1}/{len(sigmas)}, CW: {ii+1}/{d_test.shape[0]}   ', end="\r")

        x_test[ii] = polar_transform_iter(u_test[ii])
        d_test_batch = np.tile(d_test[ii], (test_batch, 1))
        x_test_batch = np.tile(x_test[ii], (test_batch, 1))


        # Modulator (BPSK)
        s_test = -2*x_test_batch + 1

        # Channel (AWGN)
        y_test = s_test + sigmas[i]*np.random.standard_normal(s_test.shape)


        nb_bits[nb_bits.shape[0]-1][ii][i] += d_test_batch.size
        nb_errors[nb_errors.shape[0]-1][ii][i] += decoder.evaluate(y_test, d_test_batch, batch_size=test_batch, verbose=0)[1]

## Take 16 random elements

### Sptit codebook on train / validation parts

In [ ]:
train_size = 16
np.random.seed(seed=1337)
idx = list(np.random.choice(2**k, size=train_size, replace=False))
print(f'training indices: {idx}')
inputs_train = inputs[idx]
codewords_train = codewords[idx]
d_test = np.delete(inputs, idx, axis=0)

### Fit model

In [ ]:
%%time
model, decoder = update_model('dense-128-64-32')
history = model.fit(codewords_train, inputs_train, batch_size=batch_size, epochs=nb_epoch,
                    shuffle=True, verbose=0, callbacks=[TqdmCallback(verbose=0)])

### Evaluate model

In [ ]:
exp_descr.append('random 16 elements')                                          # Add legend
nb_errors = np.append(nb_errors, np.zeros((1, 2**k, len(sigmas))), axis=0)     # Add row for new data
nb_bits = np.append(nb_bits, np.zeros((1, 2**k, len(sigmas))), axis=0)     # Add row for new data

for i in range(0,len(sigmas)):

    x_test = np.zeros((d_test.shape[0], N), dtype=bool)
    u_test = np.zeros((d_test.shape[0], N), dtype=bool)

    u_test[:, log_vector] = d_test
    for ii in range(0,d_test.shape[0]):

        print(f'Decoding SNR: {i+1}/{len(sigmas)}, CW: {ii+1}/{d_test.shape[0]}   ', end="\r")

        x_test[ii] = polar_transform_iter(u_test[ii])
        d_test_batch = np.tile(d_test[ii], (test_batch, 1))
        x_test_batch = np.tile(x_test[ii], (test_batch, 1))


        # Modulator (BPSK)
        s_test = -2*x_test_batch + 1

        # Channel (AWGN)
        y_test = s_test + sigmas[i]*np.random.standard_normal(s_test.shape)


        nb_bits[nb_bits.shape[0]-1][ii][i] += d_test_batch.size
        nb_errors[nb_errors.shape[0]-1][ii][i] += decoder.evaluate(y_test, d_test_batch, batch_size=test_batch, verbose=0)[1]

## Take 32 random elements

### Sptit codebook on train / validation parts

In [ ]:
train_size = 32
np.random.seed(seed=1337)
idx = list(np.random.choice(2**k, size=train_size, replace=False))
print(f'training indices: {np.sort(idx)}')
inputs_train = inputs[idx]
codewords_train = codewords[idx]
d_test = np.delete(inputs, idx, axis=0)

### Fit model

In [ ]:
%%time
model, decoder = update_model('dense-128-64-32')
history = model.fit(codewords_train, inputs_train, batch_size=batch_size, epochs=nb_epoch,
                    shuffle=True, verbose=0, callbacks=[TqdmCallback(verbose=0)])

### Evaluate model

In [ ]:
exp_descr.append('random 32 elements')                                          # Add legend
nb_errors = np.append(nb_errors, np.zeros((1, 2**k, len(sigmas))), axis=0)     # Add row for new data
nb_bits = np.append(nb_bits, np.zeros((1, 2**k, len(sigmas))), axis=0)     # Add row for new data

for i in range(0,len(sigmas)):

    x_test = np.zeros((d_test.shape[0], N), dtype=bool)
    u_test = np.zeros((d_test.shape[0], N), dtype=bool)

    u_test[:, log_vector] = d_test
    for ii in range(0,d_test.shape[0]):

        print(f'Decoding SNR: {i+1}/{len(sigmas)}, CW: {ii+1}/{d_test.shape[0]}   ', end="\r")

        x_test[ii] = polar_transform_iter(u_test[ii])
        d_test_batch = np.tile(d_test[ii], (test_batch, 1))
        x_test_batch = np.tile(x_test[ii], (test_batch, 1))


        # Modulator (BPSK)
        s_test = -2*x_test_batch + 1

        # Channel (AWGN)
        y_test = s_test + sigmas[i]*np.random.standard_normal(s_test.shape)


        nb_bits[nb_bits.shape[0]-1][ii][i] += d_test_batch.size
        nb_errors[nb_errors.shape[0]-1][ii][i] += decoder.evaluate(y_test, d_test_batch, batch_size=test_batch, verbose=0)[1]

## Take 64 random elements

### Sptit codebook on train / validation parts

In [ ]:
train_size = 64
np.random.seed(seed=1337)
idx = list(np.random.choice(2**k, size=train_size, replace=False))
print(f'training indices: {idx}')
inputs_train = inputs[idx]
codewords_train = codewords[idx]
d_test = np.delete(inputs, idx, axis=0)

### Fit model

In [ ]:
%%time
model, decoder = update_model('dense-128-64-32')
history = model.fit(codewords_train, inputs_train, batch_size=batch_size, epochs=nb_epoch,
                    shuffle=True, verbose=0, callbacks=[TqdmCallback(verbose=0)])

### Evaluate model

In [ ]:
exp_descr.append('random 64 elements')                                          # Add legend
nb_errors = np.append(nb_errors, np.zeros((1, 2**k, len(sigmas))), axis=0)     # Add row for new data
nb_bits = np.append(nb_bits, np.zeros((1, 2**k, len(sigmas))), axis=0)     # Add row for new data

for i in range(0,len(sigmas)):

    x_test = np.zeros((d_test.shape[0], N), dtype=bool)
    u_test = np.zeros((d_test.shape[0], N), dtype=bool)

    u_test[:, log_vector] = d_test
    for ii in range(0,d_test.shape[0]):

        print(f'Decoding SNR: {i+1}/{len(sigmas)}, CW: {ii+1}/{d_test.shape[0]}   ', end="\r")

        x_test[ii] = polar_transform_iter(u_test[ii])
        d_test_batch = np.tile(d_test[ii], (test_batch, 1))
        x_test_batch = np.tile(x_test[ii], (test_batch, 1))


        # Modulator (BPSK)
        s_test = -2*x_test_batch + 1

        # Channel (AWGN)
        y_test = s_test + sigmas[i]*np.random.standard_normal(s_test.shape)


        nb_bits[nb_bits.shape[0]-1][ii][i] += d_test_batch.size
        nb_errors[nb_errors.shape[0]-1][ii][i] += decoder.evaluate(y_test, d_test_batch, batch_size=test_batch, verbose=0)[1]

## Take 128 random elements

### Sptit codebook on train / validation parts

In [ ]:
train_size = 128
np.random.seed(seed=1337)
idx = list(np.random.choice(2**k, size=train_size, replace=False))
print(f'training indices: {idx}')
inputs_train = inputs[idx]
codewords_train = codewords[idx]
d_test = np.delete(inputs, idx, axis=0)

### Fit model

In [ ]:
%%time
model, decoder = update_model('dense-128-64-32')
history = model.fit(codewords_train, inputs_train, batch_size=batch_size, epochs=nb_epoch,
                    shuffle=True, verbose=0, callbacks=[TqdmCallback(verbose=0)])

### Evaluate model

In [ ]:
exp_descr.append('random 128 elements')                                          # Add legend
nb_errors = np.append(nb_errors, np.zeros((1, 2**k, len(sigmas))), axis=0)     # Add row for new data
nb_bits = np.append(nb_bits, np.zeros((1, 2**k, len(sigmas))), axis=0)     # Add row for new data

for i in range(0,len(sigmas)):

    x_test = np.zeros((d_test.shape[0], N), dtype=bool)
    u_test = np.zeros((d_test.shape[0], N), dtype=bool)

    u_test[:, log_vector] = d_test
    for ii in range(0,d_test.shape[0]):

        print(f'Decoding SNR: {i+1}/{len(sigmas)}, CW: {ii+1}/{d_test.shape[0]}   ', end="\r")

        x_test[ii] = polar_transform_iter(u_test[ii])
        d_test_batch = np.tile(d_test[ii], (test_batch, 1))
        x_test_batch = np.tile(x_test[ii], (test_batch, 1))


        # Modulator (BPSK)
        s_test = -2*x_test_batch + 1

        # Channel (AWGN)
        y_test = s_test + sigmas[i]*np.random.standard_normal(s_test.shape)


        nb_bits[nb_bits.shape[0]-1][ii][i] += d_test_batch.size
        nb_errors[nb_errors.shape[0]-1][ii][i] += decoder.evaluate(y_test, d_test_batch, batch_size=test_batch, verbose=0)[1]

## Take 86 random elements

### Sptit codebook on train / validation parts

In [ ]:
train_size = 86
np.random.seed(seed=1337)
idx = list(np.random.choice(2**k, size=train_size, replace=False))
print(f'training indices: {idx}')
inputs_train = inputs[idx]
codewords_train = codewords[idx]
d_test = np.delete(inputs, idx, axis=0)

### Fit model

In [ ]:
%%time
model, decoder = update_model('dense-128-64-32')
history = model.fit(codewords_train, inputs_train, batch_size=batch_size, epochs=nb_epoch,
                    shuffle=True, verbose=0, callbacks=[TqdmCallback(verbose=0)])

### Evaluate model

In [ ]:
exp_descr.append('random 86 elements')                                          # Add legend
nb_errors = np.append(nb_errors, np.zeros((1, 2**k, len(sigmas))), axis=0)     # Add row for new data
nb_bits = np.append(nb_bits, np.zeros((1, 2**k, len(sigmas))), axis=0)     # Add row for new data

for i in range(0,len(sigmas)):

    x_test = np.zeros((d_test.shape[0], N), dtype=bool)
    u_test = np.zeros((d_test.shape[0], N), dtype=bool)

    u_test[:, log_vector] = d_test
    for ii in range(0,d_test.shape[0]):

        print(f'Decoding SNR: {i+1}/{len(sigmas)}, CW: {ii+1}/{d_test.shape[0]}   ', end="\r")

        x_test[ii] = polar_transform_iter(u_test[ii])
        d_test_batch = np.tile(d_test[ii], (test_batch, 1))
        x_test_batch = np.tile(x_test[ii], (test_batch, 1))


        # Modulator (BPSK)
        s_test = -2*x_test_batch + 1

        # Channel (AWGN)
        y_test = s_test + sigmas[i]*np.random.standard_normal(s_test.shape)


        nb_bits[nb_bits.shape[0]-1][ii][i] += d_test_batch.size
        nb_errors[nb_errors.shape[0]-1][ii][i] += decoder.evaluate(y_test, d_test_batch, batch_size=test_batch, verbose=0)[1]

In [ ]:
plt.figure(figsize = (16, 8))
for experiment in range(0, 2):
    plt.plot(nb_errors[experiment,:,-1]/nb_bits[experiment,:,-1])
plt.legend(exp_descr, loc=3)
plt.yscale('log')
plt.xlabel('codeword index')
plt.ylabel('BER')    
plt.grid(True)
plt.title(f'{code} codes')
plt.show()

In [ ]:
ber = np.sum(nb_errors, axis=1) / np.sum(nb_bits, axis=1)

plt.figure(figsize = (16, 8))
for experiment in range(0, nb_bits.shape[0]):
    plt.plot(10*np.log10(1/(2*sigmas**2)) - 10*np.log10(k/N), ber[experiment])
plt.legend(exp_descr, loc=3)
plt.yscale('log')
plt.xlabel('$E_b/N_0$')
plt.ylabel('BER')    
plt.grid(True)
plt.title(f'{code} codes')
plt.show()

# Iterative picking

### With noise

In [ ]:
noise = True

## Train on 16 codewords, test on 240

In [ ]:
train_size = 16
idx = []
for i in range(train_size):
    idx.append(i*train_size+i)
print(f'training indices: {idx}')
inputs_train = inputs[idx]
codewords_train = codewords[idx]
d_test = np.delete(inputs, idx, axis=0)
print(f'training information words:\n{inputs_train.astype(int)}')

In [ ]:
inputs_train.shape,  d_test.shape

In [ ]:
inputs_train.astype(int)

In [ ]:
codewords_train

### Fit model

In [ ]:
%%time
model, decoder = update_model('dense-128-64-32')
history = model.fit(codewords_train, inputs_train, batch_size=batch_size, epochs=nb_epoch,
                    shuffle=True, verbose=0, callbacks=[TqdmCallback(verbose=0)])

### Evaluate model

In [ ]:
exp_descr.append('iterative picking, 16 train / 240 test')                     # Add legend
nb_errors = np.append(nb_errors, np.zeros((1, 2**k, len(sigmas))), axis=0)     # Add row for new data
nb_bits = np.append(nb_bits, np.zeros((1, 2**k, len(sigmas))), axis=0)         # Add row for new data

for i in range(0,len(sigmas)):

    x_test = np.zeros((d_test.shape[0], N), dtype=bool)
    u_test = np.zeros((d_test.shape[0], N), dtype=bool)

    u_test[:, log_vector] = d_test
    for ii in range(0,d_test.shape[0]):

        print(f'Decoding SNR: {i+1}/{len(sigmas)}, CW: {ii+1}/{d_test.shape[0]}   ', end="\r")

        x_test[ii] = polar_transform_iter(u_test[ii])
        d_test_batch = np.tile(d_test[ii], (test_batch, 1))
        x_test_batch = np.tile(x_test[ii], (test_batch, 1))


        # Modulator (BPSK)
        s_test = -2*x_test_batch + 1

        # Channel (AWGN)
        y_test = s_test + sigmas[i]*np.random.standard_normal(s_test.shape)


        nb_bits[nb_bits.shape[0]-1][ii][i] += d_test_batch.size
        nb_errors[nb_errors.shape[0]-1][ii][i] += decoder.evaluate(y_test, d_test_batch, batch_size=test_batch, verbose=0)[1]

## Train on 29 codewords, test on 227

In [ ]:
init_step_size = 8
idx = []
for i in range(29):
    idx.append(int(i*init_step_size+i))
print(f'training indices: {idx}')
inputs_train = inputs[idx]
codewords_train = codewords[idx]
d_test = np.delete(inputs, idx, axis=0)

In [ ]:
inputs_train.shape,  d_test.shape

In [ ]:
inputs_train.astype(int)

### Fit model

In [ ]:
%%time
model, decoder = update_model('dense-128-64-32')
history = model.fit(codewords_train, inputs_train, batch_size=batch_size, epochs=nb_epoch,
                    shuffle=True, verbose=0, callbacks=[TqdmCallback(verbose=0)])

### Evaluate model

In [ ]:
exp_descr.append('iterative picking, 29 train / 227 test')                     # Add legend
nb_errors = np.append(nb_errors, np.zeros((1, 2**k, len(sigmas))), axis=0)     # Add row for new data
nb_bits = np.append(nb_bits, np.zeros((1, 2**k, len(sigmas))), axis=0)         # Add row for new data

for i in range(0,len(sigmas)):

    x_test = np.zeros((d_test.shape[0], N), dtype=bool)
    u_test = np.zeros((d_test.shape[0], N), dtype=bool)

    u_test[:, log_vector] = d_test
    for ii in range(0,d_test.shape[0]):

        print(f'Decoding SNR: {i+1}/{len(sigmas)}, CW: {ii+1}/{d_test.shape[0]}   ', end="\r")

        x_test[ii] = polar_transform_iter(u_test[ii])
        d_test_batch = np.tile(d_test[ii], (test_batch, 1))
        x_test_batch = np.tile(x_test[ii], (test_batch, 1))


        # Modulator (BPSK)
        s_test = -2*x_test_batch + 1

        # Channel (AWGN)
        y_test = s_test + sigmas[i]*np.random.standard_normal(s_test.shape)


        nb_bits[nb_bits.shape[0]-1][ii][i] += d_test_batch.size
        nb_errors[nb_errors.shape[0]-1][ii][i] += decoder.evaluate(y_test, d_test_batch, batch_size=test_batch, verbose=0)[1]

## Train on 52 codewords, test on 204

In [ ]:
init_step_size = 4
idx = []
for i in range(52):
    idx.append(int(i*init_step_size+i))
print(f'training indices: {idx}')
inputs_train = inputs[idx]
codewords_train = codewords[idx]
d_test = np.delete(inputs, idx, axis=0)

In [ ]:
inputs_train.shape, d_test.shape

### Fit model

In [ ]:
%%time
model, decoder = update_model('dense-128-64-32')
history = model.fit(codewords_train, inputs_train, batch_size=batch_size, epochs=nb_epoch,
                    shuffle=True, verbose=0, callbacks=[TqdmCallback(verbose=0)])

### Evaluate model

In [ ]:
exp_descr.append('iterative picking, 52 train / 204 test')                     # Add legend
nb_errors = np.append(nb_errors, np.zeros((1, 2**k, len(sigmas))), axis=0)     # Add row for new data
nb_bits = np.append(nb_bits, np.zeros((1, 2**k, len(sigmas))), axis=0)         # Add row for new data

for i in range(0,len(sigmas)):

    x_test = np.zeros((d_test.shape[0], N), dtype=bool)
    u_test = np.zeros((d_test.shape[0], N), dtype=bool)

    u_test[:, log_vector] = d_test
    for ii in range(0,d_test.shape[0]):

        print(f'Decoding SNR: {i+1}/{len(sigmas)}, CW: {ii+1}/{d_test.shape[0]}   ', end="\r")

        x_test[ii] = polar_transform_iter(u_test[ii])
        d_test_batch = np.tile(d_test[ii], (test_batch, 1))
        x_test_batch = np.tile(x_test[ii], (test_batch, 1))


        # Modulator (BPSK)
        s_test = -2*x_test_batch + 1

        # Channel (AWGN)
        y_test = s_test + sigmas[i]*np.random.standard_normal(s_test.shape)


        nb_bits[nb_bits.shape[0]-1][ii][i] += d_test_batch.size
        nb_errors[nb_errors.shape[0]-1][ii][i] += decoder.evaluate(y_test, d_test_batch, batch_size=test_batch, verbose=0)[1]

## Train on 86 codewords, test on 170

In [ ]:
init_step_size = 2
idx = []
for i in range(86):
    idx.append(int(i*init_step_size+i))
print(f'training indices: {idx}')
inputs_train = inputs[idx]
codewords_train = codewords[idx]
d_test = np.delete(inputs, idx, axis=0)

In [ ]:
inputs_train.shape, d_test.shape

### Fit model

In [ ]:
%%time
model, decoder = update_model('dense-128-64-32')
history = model.fit(codewords_train, inputs_train, batch_size=batch_size, epochs=nb_epoch,
                    shuffle=True, verbose=0, callbacks=[TqdmCallback(verbose=0)])

### Evaluate model

In [ ]:
exp_descr.append('iterative picking, 86 train / 170 test')                     # Add legend
nb_errors = np.append(nb_errors, np.zeros((1, 2**k, len(sigmas))), axis=0)     # Add row for new data
nb_bits = np.append(nb_bits, np.zeros((1, 2**k, len(sigmas))), axis=0)         # Add row for new data

for i in range(0,len(sigmas)):

    x_test = np.zeros((d_test.shape[0], N), dtype=bool)
    u_test = np.zeros((d_test.shape[0], N), dtype=bool)

    u_test[:, log_vector] = d_test
    for ii in range(0,d_test.shape[0]):

        print(f'Decoding SNR: {i+1}/{len(sigmas)}, CW: {ii+1}/{d_test.shape[0]}   ', end="\r")

        x_test[ii] = polar_transform_iter(u_test[ii])
        d_test_batch = np.tile(d_test[ii], (test_batch, 1))
        x_test_batch = np.tile(x_test[ii], (test_batch, 1))


        # Modulator (BPSK)
        s_test = -2*x_test_batch + 1

        # Channel (AWGN)
        y_test = s_test + sigmas[i]*np.random.standard_normal(s_test.shape)


        nb_bits[nb_bits.shape[0]-1][ii][i] += d_test_batch.size
        nb_errors[nb_errors.shape[0]-1][ii][i] += decoder.evaluate(y_test, d_test_batch, batch_size=test_batch, verbose=0)[1]

## Train on 30 codewords, test on 226

In [ ]:
init_step_size = 8
idx = []
for i in range(29):
    idx.append(int(i*init_step_size+i))
print(f'training indices: {idx}')
inputs_train = inputs[idx]
codewords_train = codewords[idx]
d_test = np.delete(inputs, idx, axis=0)

In [ ]:
inputs_train = np.vstack((inputs_train, d_test[-1]))
codewords_train = np.vstack((codewords_train, codewords[-1]))
d_test = np.delete(d_test, -1, axis = 0)

In [ ]:
inputs_train.shape, codewords_train.shape, d_test.shape

### Fit model

In [ ]:
%%time
model, decoder = update_model('dense-128-64-32')
history = model.fit(codewords_train, inputs_train, batch_size=batch_size, epochs=nb_epoch,
                    shuffle=True, verbose=0, callbacks=[TqdmCallback(verbose=0)])

### Evaluate model

In [ ]:
# Same test / train codebook as in 29/227 case, but with last all-true
# codewornb_errorsrain CB
exp_descr.append('iterative picking, 30 train / 226 test')                     # Add legend
nb_errors = np.append(nb_errors, np.zeros((1, 2**k, len(sigmas))), axis=0)     # Add row for new data
nb_bits = np.append(nb_bits, np.zeros((1, 2**k, len(sigmas))), axis=0)         # Add row for new data

for i in range(0,len(sigmas)):

    x_test = np.zeros((d_test.shape[0], N), dtype=bool)
    u_test = np.zeros((d_test.shape[0], N), dtype=bool)

    u_test[:, log_vector] = d_test
    for ii in range(0,d_test.shape[0]):

        print(f'Decoding SNR: {i+1}/{len(sigmas)}, CW: {ii+1}/{d_test.shape[0]}   ', end="\r")

        x_test[ii] = polar_transform_iter(u_test[ii])
        d_test_batch = np.tile(d_test[ii], (test_batch, 1))
        x_test_batch = np.tile(x_test[ii], (test_batch, 1))


        # Modulator (BPSK)
        s_test = -2*x_test_batch + 1

        # Channel (AWGN)
        y_test = s_test + sigmas[i]*np.random.standard_normal(s_test.shape)


        nb_bits[nb_bits.shape[0]-1][ii][i] += d_test_batch.size
        nb_errors[nb_errors.shape[0]-1][ii][i] += decoder.evaluate(y_test, d_test_batch, batch_size=test_batch, verbose=0)[1]

## Plot test results

In [ ]:
exp_descr

In [ ]:
import matplotlib
matplotlib.rcParams.update({'font.size': 17})

legend = []
fig = plt.figure(figsize = (16, 8))
for experiment in [1]:
    plt.plot(nb_errors[experiment,:,-1]/nb_bits[experiment,:,-1])
    legend.append(exp_descr[experiment])
plt.vlines([0, 16, 32, 48, 64, 80, 96, 112, 128, 144, 160, 176, 192, 208, 224, 240], 0, 1, linestyles='dashed', colors='red')
# plt.legend(legend, loc=3)
plt.yscale('log')
plt.xlabel('codeword index')
plt.ylabel('BER')    
plt.xlim(0,240)
plt.ylim(1.5e-1,1)
plt.grid(True)
plt.title(f'Polar(16,8). FCNN trained on 16 subsequent cws, validated on unseen cws')
plt.show()

In [ ]:
ber = np.sum(nb_errors, axis=1) / np.sum(nb_bits, axis=1)

plt.figure(figsize = (16, 8))
for experiment in range(0, len(exp_descr)):
    plt.plot(10*np.log10(1/(2*sigmas**2)) - 10*np.log10(k/N), ber[experiment])
plt.legend(exp_descr, loc=3)
plt.yscale('log')
plt.xlabel('$E_b/N_0$')
plt.ylabel('BER')    
plt.grid(True)
plt.title(f'{code} codes')
plt.show()

In [ ]:
ber = np.sum(nb_errors, axis=1) / np.sum(nb_bits, axis=1)

plt.figure(figsize = (16, 8))
for experiment in [0, 1, 2, 3]:
    plt.plot(10*np.log10(1/(2*sigmas**2)) - 10*np.log10(k/N), ber[experiment])
plt.legend(exp_descr, loc=3)
plt.yscale('log')
plt.xlabel('$E_b/N_0$')
plt.ylabel('BER')    
plt.grid(True)
plt.title(f'{code} codes')
plt.show()

In [ ]:
ber = np.sum(nb_errors, axis=1) / np.sum(nb_bits, axis=1)
legend = []
plt.figure(figsize = (16, 8))
for experiment in [0, 1, 2, 3, 5, 6, 7, 8]:
    plt.plot(10*np.log10(1/(2*sigmas**2)) - 10*np.log10(k/N), ber[experiment])
    legend.append(exp_descr[experiment])
plt.legend(legend, loc=3)
plt.yscale('log')
plt.xlabel('$E_b/N_0$')
plt.ylabel('BER')    
plt.grid(True)
plt.title(f'{code} codes')
plt.show()

In [ ]:
ber = np.sum(nb_errors, axis=1) / np.sum(nb_bits, axis=1)

legend = []

plt.figure(figsize = (16, 8))
for experiment in [9, 10, 11, 13]:
    plt.plot(10*np.log10(1/(2*sigmas**2)) - 10*np.log10(k/N), ber[experiment])
    legend.append(exp_descr[experiment])
plt.legend(legend, loc=3)
plt.yscale('log')
plt.xlabel('$E_b/N_0$')
plt.ylabel('BER')    
plt.grid(True)
plt.title(f'{code} codes')
plt.show()

In [ ]:
ber = np.sum(nb_errors, axis=1) / np.sum(nb_bits, axis=1)

legend = []

fig = plt.figure(figsize = (16, 8))
for experiment in [5, 6, 7, -1]:
    plt.plot(10*np.log10(1/(2*sigmas**2)) - 10*np.log10(k/N), ber[experiment])
    legend.append(exp_descr[experiment])
legend = ['Train set - 16 codewords', 'Train set - 32 codewords', 'Train set - 64 codewords', 'Train set - 86 codewords']
plt.legend(legend, loc=3)
plt.yscale('log')
plt.xlabel('$E_b/N_0$')
plt.ylabel('BER')    
plt.grid(True)
# plt.title(f'{code} codes')
plt.title(f'Polar(16,8). FCNN trained on random cws, validated on unseen cws')
plt.show()